In [1]:
from src.utils import *
import sys
from tqdm import tqdm
import csv
import re
from sklearn.metrics import accuracy_score, balanced_accuracy_score

In [2]:
# Function to extract the numeric part of the filename (after the last underscore and before the file extension)
def extract_numeric_part(path):
    # This regex extracts the number after the last underscore and before the file extension (.flac)
    match = re.search(r'_(\d+)\.flac$', path)
    if match:
        return match.group(1)  # Return only the numeric part
    return None  # Return None if the pattern doesn't match


def pred_and_labels_clean(file1_csv, file2_csv):
    '''
    file1: probabilities list .csv
    file2: eval dataset .csv
    '''
    
    prediction_list = []
    label_list = []
    
    # Step 1: Read file2 into a dictionary for quick look-up based on the numeric part of the file path
    file2_dict = {}
    with open(file2_csv, mode='r') as file2:
        reader = csv.DictReader(file2)
        for row in reader:
            numeric_file2 = extract_numeric_part(row['path'])
            if numeric_file2:
                file2_dict[numeric_file2] = row['label']

    # Step 2: Traverse file1 and check against file2 dictionary, using tqdm for progress tracking
    with open(file1_csv, mode='r') as file1:
        reader = csv.DictReader(file1)
        total_rows = sum(1 for _ in open(file1_csv)) - 1  # Calculate total rows for the progress bar (excluding header)
    
        file1.seek(0)  # Reset the reader position back to the start of the file after counting
        for row in reader:
            # Extract the numeric part from file1's path
            file1_path = row['Filename']
            numeric_file1 = extract_numeric_part(file1_path)
        
            # Step 3: Check if the numeric part exists in the file2 dictionary
            if numeric_file1 and numeric_file1 in file2_dict:
                pred_class_0 = float(row['Pred.class 0'])
                pred_class_1 = float(row['Pred.class 1'])
            
                # Step 4: Append prediction based on comparison
                prediction_list.append(0 if pred_class_0 > pred_class_1 else 1)
            
                # Step 5: Append the corresponding label from file2
                label_list.append(int(file2_dict[numeric_file1]))

    
    return prediction_list, label_list




def pred_and_labels_attack(file1_csv, file2_csv):
    
    def extract_numeric_part(path):
        # This regex captures the numeric part following 'LA_E_' and stops at the next underscore or period
        match = re.search(r'LA_E_(\d+)', path)
        if match:
            return match.group(1)  # Return only the numeric part
        return None  # Return None if the pattern doesn't match
    
    prediction_list = []
    label_list = []
    
    # Step 1: Read file2 into a dictionary for quick look-up based on the numeric part of the file path
    file2_dict = {}
    with open(file2_csv, mode='r') as file2:
        reader = csv.DictReader(file2)
        for row in reader:
            numeric_file2 = extract_numeric_part(row['path'])
            if numeric_file2:
                file2_dict[numeric_file2] = row['label']

    # Step 2: Traverse file1 and check against file2 dictionary, using tqdm for progress tracking
    with open(file1_csv, mode='r') as file1:
        reader = csv.DictReader(file1)
        total_rows = sum(1 for _ in open(file1_csv)) - 1  # Calculate total rows for the progress bar (excluding header)
    
        file1.seek(0)  # Reset the reader position back to the start of the file after counting
        for row in reader:
            # Extract the numeric part from file1's path
            file1_path = row['Filename']
            numeric_file1 = extract_numeric_part(file1_path)
        
            # Step 3: Check if the numeric part exists in the file2 dictionary
            if numeric_file1 and numeric_file1 in file2_dict:
                pred_class_0 = float(row['Pred.class 0'])
                pred_class_1 = float(row['Pred.class 1'])
            
                # Step 4: Append prediction based on comparison
                prediction_list.append(0 if pred_class_0 > pred_class_1 else 1)
            
                # Step 5: Append the corresponding label from file2
                label_list.append(int(file2_dict[numeric_file1]))

    
    return prediction_list, label_list




In [3]:
def eval_clean(eval_model, model_version, type_of_spec, feature, dataset):
    
    script_dir = os.getcwd()  # get directory of current script
    print('Evaluating clean dataset...')
    probs_csv = f'probs_{eval_model}_{model_version}_clean_{dataset}_{type_of_spec}_{feature}.csv'
    #probs = pd.read_csv(os.path.join(script_dir, probs_csv), header=0, engine='python')
    
    if dataset == '3s':
        eval_csv = os.path.join(os.path.dirname(script_dir), 'data', 'df_eval_19_3s.csv' )
    else:
        eval_csv = os.path.join(os.path.dirname(script_dir), 'data', 'df_eval_19.csv' )


    pred_labels, GT_labels = pred_and_labels_clean(file1_csv=probs_csv, file2_csv=eval_csv)
    
    print(len(pred_labels))
    print(len(GT_labels))

    # UNBALANCED ACCURACY
    UA = accuracy_score(y_true=GT_labels, y_pred=pred_labels)
    BA = balanced_accuracy_score(y_true=GT_labels, y_pred=pred_labels)

    print(f'Eval model: {eval_model} {model_version}, clean dataset: {dataset}, feature = {feature} --> UA = {UA*100:.2f}%, BA = {BA*100:.2f}% ')
    

In [4]:
def eval_attack(attack, eval_model, attack_model, model_version, type_of_spec, feature, dataset, epsilon, q_res, q_sen):
    
    epsilon_str = str(epsilon).replace('.', 'dot')
    script_dir = os.getcwd()  # get directory of current script
    
    if attack != 'Ensemble' and attack != None:
        probs_csv = f'probs_{eval_model}_{model_version}_{attack}_{attack_model}_{dataset}_{epsilon_str}_{type_of_spec}_{feature}.csv'
        probs = pd.read_csv(os.path.join(script_dir, probs_csv), header=0, engine='python')
    elif attack == 'Ensemble':
        probs_csv = f'probs_{eval_model}_{model_version}_Ensemble_{dataset}_{q_res}_{q_sen}_{epsilon_str}_{type_of_spec}_{feature}.csv'
        probs = pd.read_csv(os.path.join(script_dir, probs_csv), header=0, engine='python')

    # GT labels
    if dataset == '3s':
        eval_csv = os.path.join(os.path.dirname(script_dir), 'data', 'df_eval_19_3s.csv' )
    else:
        eval_csv = os.path.join(os.path.dirname(script_dir), 'data', 'df_eval_19.csv' )
        

    pred_labels, GT_labels = pred_and_labels_attack(file1_csv=probs_csv, file2_csv=eval_csv)
    
    #print(len(pred_labels))
    #print(len(GT_labels))

    # UNBALANCED ACCURACY
    UA = accuracy_score(y_true=GT_labels, y_pred=pred_labels)
    BA = balanced_accuracy_score(y_true=GT_labels, y_pred=pred_labels)
    
    if attack != 'Ensemble':    
        print(f'Eval model: {eval_model} {model_version}, attack: {attack}, attack model: {attack_model} {model_version}, dataset: {dataset}, eps={epsilon}, feature = {feature} --> UA = {UA*100:.2f}%, BA = {BA*100:.2f}% ')
    elif attack == 'Ensemble':
        print(f'Eval model: {eval_model} {model_version}, attack: Ensemble, q_res = {q_res}, q_sen = {q_sen}, dataset: {dataset}, eps={epsilon}, feature = {feature} --> UA = {UA*100:.2f}%, BA = {BA*100:.2f}% ')

***
## Clean dataset with magnitude

In [13]:
eval_clean(eval_model='ResNet', model_version='v0', type_of_spec='mag', feature='audio', dataset='whole')

Evaluating clean dataset...
71237
71237
Eval model: ResNet v0, clean dataset: whole, feature = audio --> UA = 89.45%, BA = 49.89% 


***
## Clean dataset

In [5]:
eval_clean(eval_model='ResNet', model_version='v0', type_of_spec='pow', feature='audio', dataset='whole')

Evaluating clean dataset...
71237
71237
Eval model: ResNet v0, clean dataset: whole, feature = audio --> UA = 89.45%, BA = 49.89% 


In [16]:
eval_clean(eval_model='SENet', model_version='v0', type_of_spec='pow', feature='audio', dataset='whole')

Evaluating clean dataset...
71237
71237
Eval model: SENet v0, clean dataset: whole, feature = audio --> UA = 70.80%, BA = 82.62% 


***
## Whole dataset

In [11]:
eval_attack(attack='FGSM', eval_model='ResNet', attack_model='ResNet', model_version='v0', type_of_spec='pow', feature='audio', dataset='whole', epsilon=3.0, q_res=None, q_sen=None)
eval_attack(attack='FGSM', eval_model='ResNet', attack_model='ResNet', model_version='v0', type_of_spec='pow', feature='spec', dataset='whole', epsilon=3.0, q_res=None, q_sen=None)

Eval model: ResNet v0, attack: FGSM, attack model: ResNet v0, dataset: whole, eps=3.0, feature = audio --> UA = 1.39%, BA = 0.82% 
Eval model: ResNet v0, attack: FGSM, attack model: ResNet v0, dataset: whole, eps=3.0, feature = spec --> UA = 0.00%, BA = 0.00% 


In [12]:
eval_attack(attack='FGSM', eval_model='SENet', attack_model='SENet', model_version='v0', type_of_spec='pow', feature='audio', dataset='whole', epsilon=3.0, q_res=None, q_sen=None)
eval_attack(attack='FGSM', eval_model='SENet', attack_model='SENet', model_version='v0', type_of_spec='pow', feature='spec', dataset='whole', epsilon=3.0, q_res=None, q_sen=None)

Eval model: SENet v0, attack: FGSM, attack model: SENet v0, dataset: whole, eps=3.0, feature = audio --> UA = 13.50%, BA = 25.57% 
Eval model: SENet v0, attack: FGSM, attack model: SENet v0, dataset: whole, eps=3.0, feature = spec --> UA = 9.24%, BA = 5.15% 
